In [1]:
spark.sql("set spark.sql.streaming.schemaInference=true")

StatementMeta(DataProcess31, 0, 1, Finished, Available)

DataFrame[key: string, value: string]

In [2]:
df_stream = spark \
    .readStream \
    .format('parquet') \
    .load('abfss://raw@adlsopenclassanalytics.dfs.core.windows.net/Cornerchy_OLTP/Sales/*/*/*/*.parquet')

StatementMeta(DataProcess, 29, 2, Finished, Available)

In [3]:
df_stream.isStreaming

StatementMeta(DataProcess, 29, 3, Finished, Available)

True

In [2]:
from delta import *

StatementMeta(DataProcess31, 0, 2, Finished, Available)

In [5]:
df_proceess = df_stream.writeStream.format("delta") \
    .option('checkpointLocation','abfss://std@adlsopenclassanalytics.dfs.core.windows.net/Cornerchy/Sales_demo02_checkpointLocation') \
    .option('path','abfss://std@adlsopenclassanalytics.dfs.core.windows.net/Cornerchy/Sales_demo02/') \
    .start()

StatementMeta(DataProcess, 29, 5, Finished, Available)

In [9]:
df_proceess.status

StatementMeta(DataProcess, 29, 9, Finished, Available)

{'message': 'Stopped', 'isDataAvailable': False, 'isTriggerActive': False}

In [11]:
df_proceess.stop()

StatementMeta(DataProcess, 29, 11, Finished, Available)

In [3]:
from pyspark.sql.functions import substring, isnull, col, regexp_replace, current_timestamp, lit, month, dayofmonth, year, to_date, lpad, input_file_name, coalesce 

StatementMeta(DataProcess31, 0, 3, Finished, Available)

In [12]:
maxFilesPerTrigger = 1
path_source_completed = 'abfss://raw@adlsopenclassanalytics.dfs.core.windows.net/Cornerchy_OLTP/Sales/*/*/*/*.parquet'
primarykey = 'salelineid'
metadata_columns = 'load_date, type_process'
typeProcess = 'inc'
partitionBy = None
level = 'month'
path_destination_completed = 'abfss://std@adlsopenclassanalytics.dfs.core.windows.net/Cornerchy/Sales_demo02/'
repartition_num_files = 32
path_deltaCheckpointPath = 'abfss://std@adlsopenclassanalytics.dfs.core.windows.net/Cornerchy/Sales_demo02_checkpointPath/'

StatementMeta(DataProcess31, 0, 12, Finished, Available)

In [5]:
def allColumns(df, metadata_columns):
    return [column for column in df.columns if column not in metadata_columns]
    
def generatePKs(primaryKeys):
    pks_dict = dict()
    for index, value in enumerate(primaryKeys):
        pks_dict[f"deltaTable.{value}"] = f"incTable.{value}"
    return str(pks_dict).replace("{", "").replace("}","").replace("'","").replace(": ", " <=> ").replace(",", " and")

def checkDeltaFolder(path):
    try:
        DeltaTable.forPath(spark, path)
        return True
    except Exception as e:
        return False

StatementMeta(DataProcess31, 0, 5, Finished, Available)

In [7]:
def upsertToDelta(microBatchOutputDF, batchId, primarykey, metadata_columns, partitionBy, level, path_destination_completed, repartition_num_files, typeProcess):
    if primarykey == None or primarykey == "":
        columns = allColumns(microBatchOutputDF, metadata_columns.split(','))
        primaryKeys = generatePKs(columns)
    else:
        columns = primarykey.split(",")
        primaryKeys = generatePKs(columns)
        microBatchOutputDF = microBatchOutputDF.withColumn("load_date", current_timestamp())
        microBatchOutputDF = microBatchOutputDF.withColumn("type_process", lit(typeProcess))
    
    if checkDeltaFolder(path_destination_completed):
        print (f"upsertToDelta: {batchId} - Merge")
        curatedTable = DeltaTable.forPath(spark, path_destination_completed)
        curatedTable.alias("deltaTable").merge(
            microBatchOutputDF.alias("incTable"),primaryKeys) \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll() \
        .execute()
    else:
        print (f"upsertToDelta: {batchId} - Full")
        if partitionBy == None or partitionBy == "":
            microBatchOutputDF.write.format("delta").mode("overwrite").option("escape","\"").save(path_destination_completed)

StatementMeta(DataProcess31, 0, 7, Finished, Available)

In [10]:
# Opción trigger one para versiones de spark a partir de 3.3 y procesamiento por lotes
df_proceess = (spark.readStream.option("maxFilesPerTrigger", maxFilesPerTrigger)
    .option("maxFileAge", 0).format('parquet').option("inferschema", True)
    .load(path_source_completed).withColumn("filename", input_file_name())
    .writeStream.format('delta')
    .trigger(availableNow=True) \
    .foreachBatch(lambda microBatchOutputDF, batchId: upsertToDelta(microBatchOutputDF, batchId, primarykey, metadata_columns, partitionBy, level, path_destination_completed, repartition_num_files, typeProcess))
    .outputMode("update")
    .option('checkpointLocation',path_deltaCheckpointPath) \
    .start() \
    .awaitTermination()
)

StatementMeta(DataProcess31, 0, 10, Finished, Available)

upsertToDelta: 11 - Merge


In [ ]:
df_proceess.status

StatementMeta(, , , Cancelled, )

In [25]:
df_proceess.stop()

StatementMeta(DataProcess, 29, 25, Finished, Available)

AttributeError: 'NoneType' object has no attribute 'stop'

In [13]:
# Opción para simular trigger one verificando el estado del streaming
df_proceess = (spark.readStream.option("maxFilesPerTrigger", maxFilesPerTrigger)
    .option("maxFileAge", 0).format('parquet').option("inferschema", True)
    .load(path_source_completed).withColumn("filename", input_file_name())
    .writeStream.format('delta')
    .foreachBatch(lambda microBatchOutputDF, batchId: upsertToDelta(microBatchOutputDF, batchId, primarykey, metadata_columns, partitionBy, level, path_destination_completed, repartition_num_files, typeProcess))
    .outputMode("update")
    .option('checkpointLocation',path_deltaCheckpointPath) \
    .start()
)

import time
print(df_proceess.isActive)
print(df_proceess.status)
time.sleep(30)

while df_proceess.status["isDataAvailable"]:
    print("wait for load stream...")
    time.sleep(30)
print("stopping stream...")
df_proceess.stop()

StatementMeta(DataProcess31, 0, 13, Finished, Available)

True
{'message': 'Initializing sources', 'isDataAvailable': False, 'isTriggerActive': False}
stopping stream...
